In [6]:
import datetime
import sys
import os

import IPython
import IPython.display
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input, Model
from cond_rnn import ConditionalRNN
from sklearn.preprocessing import MinMaxScaler
from pmdarima.arima import auto_arima

import src.preprocessing
from src.functions import load_data, TimeSeriesTensor, create_evaluation_df, series_to_supervised, plot_train_history, validation

np.set_printoptions(threshold=sys.maxsize)

In [7]:
GEP1 = pd.read_csv('../data/GEP/Consumption_15min.csv', index_col=0, header=0, names=['value'])
GEP2 = pd.read_csv('../data/GEP/B2_Consumption_15min.csv', index_col=0, header=0, names=['value'])
GEP4 = pd.read_csv('../data/GEP/B4_Consumption_15min.csv', index_col=0, header=0, names=['value'])
datasets = [GEP1, GEP4]
names = ['GEP1', 'GEP4']

In [12]:
class Baseline(tf.keras.Model):
    def __init__(self, label_index=None):
        super().__init__()
        self.label_index = label_index

    def call(self, inputs):
        if self.label_index is None:
            return inputs
        result = inputs[:, :, self.label_index]
        return result[:, :, tf.newaxis]

In [15]:
metrics = pd.DataFrame(columns=['mae','mape', 'rmse', 'B'], index=range(len(datasets)))
from progressbar import ProgressBar
pbar = ProgressBar()
for i,df in enumerate(datasets):
    train_inputs, test_inputs, y_scaler = MIMO_fulldata_preparation(df)
    baseline = Baseline(label_index = 0)
    baseline.compile(loss=tf.losses.MeanSquaredError(), metrics=[tf.metrics.MeanSquaredError()])
    predictions_B = baseline.predict(test_inputs['X'])
    eval_df_B = create_evaluation_df(predictions_B.reshape(-1,24), test_inputs, 24, y_scaler)
    mae = validation(eval_df_B['prediction'], eval_df_B['actual'], 'MAE')
    mape = validation(eval_df_B['prediction'], eval_df_B['actual'], 'MAPE')
    rmse = validation(eval_df_B['prediction'], eval_df_B['actual'], 'RMSE')
    metrics.loc[i] = pd.Series({'mae':mae, 'mape':mape, 'rmse':rmse, 'B': names[i]})
metrics.to_csv('./results/GEP/baseline.csv')

In [ ]:
MAX_EPOCHS = 100
BATCHSIZE = 32
patience = 10
HORIZON = 24

FULL_LSTMIMO = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, input_shape=(24, 15)),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(HORIZON)
])

metrics = pd.DataFrame(columns=['mae','mape', 'rmse', 'B'], index=range(len(datasets)))
for i,df in enumerate(datasets):
    train_longer_inputs, test_longer_inputs, y_scaler = MIMO_fulldata_preparation(df)
    # full data LSTM MIMO compilation and fit
    FULL_LSTMIMO.compile(optimizer=tf.optimizers.Adam(), loss='mse', metrics=[tf.metrics.MeanSquaredError()])

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
    for j in range(1,11):
        print('RUN:{}'.format(j))
        history = FULL_LSTMIMO.fit(train_longer_inputs['X'], train_longer_inputs['target'], batch_size=32, epochs=MAX_EPOCHS,
                      validation_split=0.15,
                      callbacks=[early_stopping], verbose=1)

        FD_predictions = FULL_LSTMIMO.predict(test_longer_inputs['X'])
        FD_eval_df = create_evaluation_df(FD_predictions, test_longer_inputs, HORIZON, y_scaler)
        mae = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAE')
        mape = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAPE')
        rmse = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'RMSE')
        metrics.loc[10*i + j] = pd.Series({'mae':mae, 'mape':mape, 'rmse':rmse, 'B': names[i]})
metrics.to_csv('./results/GEP/fulldata10_LSTM.csv')

RUN:1
Epoch 1/100
116/116 [==============================] - 3s 22ms/step - loss: 0.0664 - mean_squared_error: 0.0664 - val_loss: 0.0370 - val_mean_squared_error: 0.0370
Epoch 2/100
116/116 [==============================] - 2s 18ms/step - loss: 0.0294 - mean_squared_error: 0.0294 - val_loss: 0.0270 - val_mean_squared_error: 0.0270
Epoch 3/100
116/116 [==============================] - 2s 18ms/step - loss: 0.0235 - mean_squared_error: 0.0235 - val_loss: 0.0230 - val_mean_squared_error: 0.0230
Epoch 4/100
116/116 [==============================] - 2s 16ms/step - loss: 0.0212 - mean_squared_error: 0.0212 - val_loss: 0.0226 - val_mean_squared_error: 0.0226
Epoch 5/100
116/116 [==============================] - 2s 16ms/step - loss: 0.0199 - mean_squared_error: 0.0199 - val_loss: 0.0207 - val_mean_squared_error: 0.0207
Epoch 6/100
116/116 [==============================] - 2s 16ms/step - loss: 0.0189 - mean_squared_error: 0.0189 - val_loss: 0.0207 - val_mean_squared_error: 0.0207
Epoch 7/10

Epoch 24/100
116/116 [==============================] - 2s 15ms/step - loss: 0.0050 - mean_squared_error: 0.0050 - val_loss: 0.0344 - val_mean_squared_error: 0.0344
Epoch 25/100
116/116 [==============================] - 1s 13ms/step - loss: 0.0047 - mean_squared_error: 0.0047 - val_loss: 0.0320 - val_mean_squared_error: 0.0320
Epoch 26/100
116/116 [==============================] - 2s 14ms/step - loss: 0.0047 - mean_squared_error: 0.0047 - val_loss: 0.0325 - val_mean_squared_error: 0.0325
Epoch 27/100
116/116 [==============================] - 2s 13ms/step - loss: 0.0048 - mean_squared_error: 0.0048 - val_loss: 0.0324 - val_mean_squared_error: 0.0324
Epoch 28/100
116/116 [==============================] - 2s 14ms/step - loss: 0.0047 - mean_squared_error: 0.0047 - val_loss: 0.0353 - val_mean_squared_error: 0.0353
Epoch 29/100
116/116 [==============================] - 2s 16ms/step - loss: 0.0046 - mean_squared_error: 0.0046 - val_loss: 0.0333 - val_mean_squared_error: 0.0333
Epoch 30/1

Epoch 15/100
116/116 [==============================] - 1s 13ms/step - loss: 0.0033 - mean_squared_error: 0.0033 - val_loss: 0.0327 - val_mean_squared_error: 0.0327
Epoch 16/100
116/116 [==============================] - 2s 13ms/step - loss: 0.0034 - mean_squared_error: 0.0034 - val_loss: 0.0307 - val_mean_squared_error: 0.0307
Epoch 17/100
116/116 [==============================] - 1s 12ms/step - loss: 0.0040 - mean_squared_error: 0.0040 - val_loss: 0.0337 - val_mean_squared_error: 0.0337
Epoch 18/100
116/116 [==============================] - 2s 14ms/step - loss: 0.0032 - mean_squared_error: 0.0032 - val_loss: 0.0304 - val_mean_squared_error: 0.0304
Epoch 19/100
116/116 [==============================] - 2s 13ms/step - loss: 0.0030 - mean_squared_error: 0.0030 - val_loss: 0.0327 - val_mean_squared_error: 0.0327
Epoch 20/100
116/116 [==============================] - 2s 15ms/step - loss: 0.0031 - mean_squared_error: 0.0031 - val_loss: 0.0328 - val_mean_squared_error: 0.0328
Epoch 21/1

Epoch 16/100
116/116 [==============================] - 2s 15ms/step - loss: 0.0024 - mean_squared_error: 0.0024 - val_loss: 0.0325 - val_mean_squared_error: 0.0325
Epoch 17/100
116/116 [==============================] - 2s 17ms/step - loss: 0.0026 - mean_squared_error: 0.0026 - val_loss: 0.0324 - val_mean_squared_error: 0.0324
Epoch 18/100
116/116 [==============================] - 2s 16ms/step - loss: 0.0028 - mean_squared_error: 0.0028 - val_loss: 0.0311 - val_mean_squared_error: 0.0311
RUN:9
Epoch 1/100
116/116 [==============================] - 2s 15ms/step - loss: 0.0037 - mean_squared_error: 0.0037 - val_loss: 0.0326 - val_mean_squared_error: 0.0326
Epoch 2/100
116/116 [==============================] - 2s 13ms/step - loss: 0.0024 - mean_squared_error: 0.0024 - val_loss: 0.0341 - val_mean_squared_error: 0.0341
Epoch 3/100
116/116 [==============================] - 2s 14ms/step - loss: 0.0024 - mean_squared_error: 0.0024 - val_loss: 0.0329 - val_mean_squared_error: 0.0329
Epoch 4

Epoch 12/100
116/116 [==============================] - 2s 16ms/step - loss: 0.0039 - mean_squared_error: 0.0039 - val_loss: 0.0135 - val_mean_squared_error: 0.0135
Epoch 13/100
116/116 [==============================] - 2s 17ms/step - loss: 0.0039 - mean_squared_error: 0.0039 - val_loss: 0.0127 - val_mean_squared_error: 0.0127
Epoch 14/100
116/116 [==============================] - 2s 14ms/step - loss: 0.0038 - mean_squared_error: 0.0038 - val_loss: 0.0131 - val_mean_squared_error: 0.0131
Epoch 15/100
116/116 [==============================] - 2s 16ms/step - loss: 0.0037 - mean_squared_error: 0.0037 - val_loss: 0.0126 - val_mean_squared_error: 0.0126
RUN:2
Epoch 1/100
116/116 [==============================] - 2s 14ms/step - loss: 0.0036 - mean_squared_error: 0.0036 - val_loss: 0.0120 - val_mean_squared_error: 0.0120
Epoch 2/100
116/116 [==============================] - 2s 14ms/step - loss: 0.0036 - mean_squared_error: 0.0036 - val_loss: 0.0126 - val_mean_squared_error: 0.0126
Epoch 

Epoch 7/100
116/116 [==============================] - 1s 12ms/step - loss: 0.0022 - mean_squared_error: 0.0022 - val_loss: 0.0145 - val_mean_squared_error: 0.0145
Epoch 8/100
116/116 [==============================] - 1s 12ms/step - loss: 0.0023 - mean_squared_error: 0.0023 - val_loss: 0.0138 - val_mean_squared_error: 0.0138
Epoch 9/100
116/116 [==============================] - 2s 14ms/step - loss: 0.0023 - mean_squared_error: 0.0023 - val_loss: 0.0148 - val_mean_squared_error: 0.0148
Epoch 10/100
116/116 [==============================] - 2s 14ms/step - loss: 0.0023 - mean_squared_error: 0.0023 - val_loss: 0.0147 - val_mean_squared_error: 0.0147
Epoch 11/100
116/116 [==============================] - 2s 16ms/step - loss: 0.0023 - mean_squared_error: 0.0023 - val_loss: 0.0151 - val_mean_squared_error: 0.0151
Epoch 12/100
116/116 [==============================] - 2s 14ms/step - loss: 0.0023 - mean_squared_error: 0.0023 - val_loss: 0.0145 - val_mean_squared_error: 0.0145
Epoch 13/100


Epoch 7/100
116/116 [==============================] - 1s 12ms/step - loss: 0.0072 - mean_squared_error: 0.0072 - val_loss: 0.0193 - val_mean_squared_error: 0.0193
Epoch 8/100
116/116 [==============================] - 2s 14ms/step - loss: 0.0066 - mean_squared_error: 0.0066 - val_loss: 0.0154 - val_mean_squared_error: 0.0154
Epoch 9/100
116/116 [==============================] - 1s 13ms/step - loss: 0.0061 - mean_squared_error: 0.0061 - val_loss: 0.0183 - val_mean_squared_error: 0.0183
Epoch 10/100
116/116 [==============================] - 2s 13ms/step - loss: 0.0057 - mean_squared_error: 0.0057 - val_loss: 0.0195 - val_mean_squared_error: 0.0195
Epoch 11/100
116/116 [==============================] - 2s 15ms/step - loss: 0.0052 - mean_squared_error: 0.0052 - val_loss: 0.0170 - val_mean_squared_error: 0.0170
RUN:2
Epoch 1/100
116/116 [==============================] - 2s 16ms/step - loss: 0.0049 - mean_squared_error: 0.0049 - val_loss: 0.0157 - val_mean_squared_error: 0.0157
Epoch 2/

In [10]:
NUM_CELLS = 32
MAX_EPOCHS = 100
BATCHSIZE = 32
patience = 10
HORIZON = 24
# ConditionalRNN with the functional API.
#i = Input(shape=[HORIZON, 5], name='time-series')
#c1 = Input(shape=[8], name='one-hot')
#c2 = Input(shape=[2], name='lags-as-features')
# add the condition tensor here.
#x = ConditionalRNN(NUM_CELLS, cell='LSTM', name='cond_rnn_0')([i, c1, c2])
# Dense layer to output
#x = Dense(HORIZON)(x)
#ConditionalRNN = Model(inputs=[i, c1, c2], outputs=[x])

ConditionalRNN = Sequential(layers=[ConditionalRNN(32, cell='LSTM'),
                                    Dense(HORIZON)])

metrics = pd.DataFrame(columns=['mae','mape', 'rmse', 'B'], index=range(len(datasets)))
from progressbar import ProgressBar
pbar = ProgressBar()
for i,df in enumerate(datasets):
    train_inputs, test_inputs, train_c1, test_c1, train_c2, test_c2, y_scaler = ConditionalRNN_data_preparation(df)
    # full data LSTM MIMO compilation and fit
    ConditionalRNN.compile(optimizer=tf.optimizers.Adam(), loss='mse', metrics=[tf.metrics.MeanSquaredError()])
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
    for j in range(1,11):
        history = ConditionalRNN.fit([train_inputs['X'],train_c1,train_c2], train_inputs['target'], batch_size=32, epochs=MAX_EPOCHS,
                      validation_split=0.15,
                      callbacks=[early_stopping], verbose=1)

        #save_model(ConditionalRNN, i)
        predictions = ConditionalRNN.predict([test_inputs['X'],test_c1,test_c2])
        eval_df = create_evaluation_df(predictions, test_inputs, HORIZON, y_scaler)
        mae = validation(eval_df['prediction'], eval_df['actual'], 'MAE')
        mape = validation(eval_df['prediction'], eval_df['actual'], 'MAPE')
        rmse = validation(eval_df['prediction'], eval_df['actual'], 'RMSE')
    #print('rmse {}'.format(rmse))
        metrics.loc[10*i + j] = pd.Series({'mae':mae, 'mape':mape, 'rmse':rmse, 'B': names[i]})
metrics.to_csv('./results/GEP/ConditionalRNN10_sequential.csv')

Epoch 1/100
Consider rewriting this model with the Functional API.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by pass

Epoch 35/100
116/116 [==============================] - 1s 13ms/step - loss: 0.0135 - mean_squared_error: 0.0135 - val_loss: 0.0169 - val_mean_squared_error: 0.0169
Epoch 36/100
116/116 [==============================] - 2s 14ms/step - loss: 0.0150 - mean_squared_error: 0.0150 - val_loss: 0.0166 - val_mean_squared_error: 0.0166
Consider rewriting this model with the Functional API.
Epoch 1/100
116/116 [==============================] - 2s 15ms/step - loss: 0.0134 - mean_squared_error: 0.0134 - val_loss: 0.0170 - val_mean_squared_error: 0.0170
Epoch 2/100
116/116 [==============================] - 2s 14ms/step - loss: 0.0130 - mean_squared_error: 0.0130 - val_loss: 0.0165 - val_mean_squared_error: 0.0165
Epoch 3/100
116/116 [==============================] - 2s 14ms/step - loss: 0.0129 - mean_squared_error: 0.0129 - val_loss: 0.0170 - val_mean_squared_error: 0.0170
Epoch 4/100
116/116 [==============================] - 2s 13ms/step - loss: 0.0127 - mean_squared_error: 0.0127 - val_loss:

Epoch 13/100
116/116 [==============================] - 1s 12ms/step - loss: 0.0063 - mean_squared_error: 0.0063 - val_loss: 0.0242 - val_mean_squared_error: 0.0242
Epoch 14/100
116/116 [==============================] - 1s 13ms/step - loss: 0.0063 - mean_squared_error: 0.0063 - val_loss: 0.0226 - val_mean_squared_error: 0.0226
Epoch 15/100
116/116 [==============================] - 1s 13ms/step - loss: 0.0066 - mean_squared_error: 0.0066 - val_loss: 0.0210 - val_mean_squared_error: 0.0210
Epoch 16/100
116/116 [==============================] - 1s 13ms/step - loss: 0.0064 - mean_squared_error: 0.0064 - val_loss: 0.0220 - val_mean_squared_error: 0.0220
Epoch 17/100
116/116 [==============================] - 1s 13ms/step - loss: 0.0061 - mean_squared_error: 0.0061 - val_loss: 0.0225 - val_mean_squared_error: 0.0225
Epoch 18/100
116/116 [==============================] - 1s 13ms/step - loss: 0.0076 - mean_squared_error: 0.0076 - val_loss: 0.0223 - val_mean_squared_error: 0.0223
Epoch 19/1

116/116 [==============================] - 2s 20ms/step - loss: 0.0179 - mean_squared_error: 0.0179 - val_loss: 0.0117 - val_mean_squared_error: 0.0117
Epoch 2/100
116/116 [==============================] - 2s 16ms/step - loss: 0.0105 - mean_squared_error: 0.0105 - val_loss: 0.0113 - val_mean_squared_error: 0.0113
Epoch 3/100
116/116 [==============================] - 2s 17ms/step - loss: 0.0093 - mean_squared_error: 0.0093 - val_loss: 0.0109 - val_mean_squared_error: 0.0109
Epoch 4/100
116/116 [==============================] - 2s 17ms/step - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.0104 - val_mean_squared_error: 0.0104
Epoch 5/100
116/116 [==============================] - 2s 16ms/step - loss: 0.0081 - mean_squared_error: 0.0081 - val_loss: 0.0102 - val_mean_squared_error: 0.0102
Epoch 6/100
116/116 [==============================] - 2s 15ms/step - loss: 0.0077 - mean_squared_error: 0.0077 - val_loss: 0.0103 - val_mean_squared_error: 0.0103
Epoch 7/100
116/116 [=======

Epoch 22/100
116/116 [==============================] - 2s 14ms/step - loss: 0.0035 - mean_squared_error: 0.0035 - val_loss: 0.0099 - val_mean_squared_error: 0.0099
Epoch 23/100
116/116 [==============================] - 2s 15ms/step - loss: 0.0034 - mean_squared_error: 0.0034 - val_loss: 0.0102 - val_mean_squared_error: 0.0102
Epoch 24/100
116/116 [==============================] - 2s 16ms/step - loss: 0.0033 - mean_squared_error: 0.0033 - val_loss: 0.0096 - val_mean_squared_error: 0.0096
Epoch 1/100
116/116 [==============================] - 2s 15ms/step - loss: 0.0033 - mean_squared_error: 0.0033 - val_loss: 0.0095 - val_mean_squared_error: 0.0095
Epoch 2/100
116/116 [==============================] - 2s 15ms/step - loss: 0.0033 - mean_squared_error: 0.0033 - val_loss: 0.0107 - val_mean_squared_error: 0.0107
Epoch 3/100
116/116 [==============================] - 2s 16ms/step - loss: 0.0033 - mean_squared_error: 0.0033 - val_loss: 0.0101 - val_mean_squared_error: 0.0101
Epoch 4/100
1

Epoch 3/100
116/116 [==============================] - 1s 12ms/step - loss: 0.0030 - mean_squared_error: 0.0030 - val_loss: 0.0098 - val_mean_squared_error: 0.0098
Epoch 4/100
116/116 [==============================] - 2s 14ms/step - loss: 0.0029 - mean_squared_error: 0.0029 - val_loss: 0.0106 - val_mean_squared_error: 0.0106
Epoch 5/100
116/116 [==============================] - 2s 15ms/step - loss: 0.0030 - mean_squared_error: 0.0030 - val_loss: 0.0106 - val_mean_squared_error: 0.0106
Epoch 6/100
116/116 [==============================] - 1s 10ms/step - loss: 0.0030 - mean_squared_error: 0.0030 - val_loss: 0.0102 - val_mean_squared_error: 0.0102
Epoch 7/100
116/116 [==============================] - 1s 10ms/step - loss: 0.0028 - mean_squared_error: 0.0028 - val_loss: 0.0102 - val_mean_squared_error: 0.0102
Epoch 8/100
116/116 [==============================] - 1s 13ms/step - loss: 0.0029 - mean_squared_error: 0.0029 - val_loss: 0.0104 - val_mean_squared_error: 0.0104
Epoch 9/100
116/

116/116 [==============================] - 2s 14ms/step - loss: 0.0052 - mean_squared_error: 0.0052 - val_loss: 0.0209 - val_mean_squared_error: 0.0209
Epoch 3/100
116/116 [==============================] - 2s 14ms/step - loss: 0.0050 - mean_squared_error: 0.0050 - val_loss: 0.0185 - val_mean_squared_error: 0.0185
Epoch 4/100
116/116 [==============================] - 2s 14ms/step - loss: 0.0049 - mean_squared_error: 0.0049 - val_loss: 0.0209 - val_mean_squared_error: 0.0209
Epoch 5/100
116/116 [==============================] - 2s 15ms/step - loss: 0.0050 - mean_squared_error: 0.0050 - val_loss: 0.0221 - val_mean_squared_error: 0.0221
Epoch 6/100
116/116 [==============================] - 2s 15ms/step - loss: 0.0051 - mean_squared_error: 0.0051 - val_loss: 0.0203 - val_mean_squared_error: 0.0203
Epoch 7/100
116/116 [==============================] - 2s 14ms/step - loss: 0.0051 - mean_squared_error: 0.0051 - val_loss: 0.0212 - val_mean_squared_error: 0.0212
Epoch 8/100
116/116 [=======

Epoch 2/100
116/116 [==============================] - 2s 14ms/step - loss: 0.0039 - mean_squared_error: 0.0039 - val_loss: 0.0206 - val_mean_squared_error: 0.0206
Epoch 3/100
116/116 [==============================] - 2s 14ms/step - loss: 0.0037 - mean_squared_error: 0.0037 - val_loss: 0.0191 - val_mean_squared_error: 0.0191
Epoch 4/100
116/116 [==============================] - 2s 13ms/step - loss: 0.0035 - mean_squared_error: 0.0035 - val_loss: 0.0211 - val_mean_squared_error: 0.0211
Epoch 5/100
116/116 [==============================] - 2s 14ms/step - loss: 0.0037 - mean_squared_error: 0.0037 - val_loss: 0.0193 - val_mean_squared_error: 0.0193
Epoch 6/100
116/116 [==============================] - 2s 15ms/step - loss: 0.0040 - mean_squared_error: 0.0040 - val_loss: 0.0189 - val_mean_squared_error: 0.0189
Epoch 7/100
116/116 [==============================] - 2s 14ms/step - loss: 0.0041 - mean_squared_error: 0.0041 - val_loss: 0.0193 - val_mean_squared_error: 0.0193
Epoch 8/100
116/

In [11]:
MAX_EPOCHS = 100
BATCHSIZE = 32
patience = 10
HORIZON = 24

FULL_LSTMIMO = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, input_shape=(24, 16)),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(HORIZON)
])

metrics = pd.DataFrame(columns=['mae','mape', 'rmse', 'B'], index=range(len(datasets)))
for i,df in enumerate(datasets[:-1]):
    train_longer_inputs, test_longer_inputs, y_scaler = MIMO_fulldataT_preparation(df)
    # full data LSTM MIMO compilation and fit
    FULL_LSTMIMO.compile(optimizer=tf.optimizers.Adam(), loss='mse', metrics=[tf.metrics.MeanSquaredError()])

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
    for j in range(1,11):
        print('RUN:{}'.format(j))
        history = FULL_LSTMIMO.fit(train_longer_inputs['X'], train_longer_inputs['target'], batch_size=32, epochs=MAX_EPOCHS,
                      validation_split=0.15,
                      callbacks=[early_stopping], verbose=1)

        FD_predictions = FULL_LSTMIMO.predict(test_longer_inputs['X'])
        FD_eval_df = create_evaluation_df(FD_predictions, test_longer_inputs, HORIZON, y_scaler)
        mae = validation(eval_df['prediction'], eval_df['actual'], 'MAE')
        mape = validation(eval_df['prediction'], eval_df['actual'], 'MAPE')
        rmse = validation(eval_df['prediction'], eval_df['actual'], 'RMSE')
        metrics.loc[10*i + j] = pd.Series({'mae':mae, 'mape':mape, 'rmse':rmse, 'B': names[i]})
metrics.to_csv('./results/GEP/Temp_LSTM.csv')

NameError: name 'temp' is not defined

In [64]:
MAX_EPOCHS = 100
BATCHSIZE = 32
patience = 10
HORIZON = 24

LSTMIMO = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, input_shape=(24, 5)),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(HORIZON)
])

metrics = pd.DataFrame(columns=['mae','rmse', 'B'], index=range(len(datasets)))
from progressbar import ProgressBar
pbar = ProgressBar()
for i,df in enumerate(datasets):
    train_inputs, test_inputs, _, _, _, _, y_scaler = ConditionalRNN_data_preparation(df)
    # full data LSTM MIMO compilation and fit
    LSTMIMO.compile(optimizer=tf.optimizers.Adam(), loss='mse', metrics=[tf.metrics.MeanSquaredError()])

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
    
    for j in range(1,11):
        history = LSTMIMO.fit(train_inputs['X'], train_inputs['target'], batch_size=32, epochs=MAX_EPOCHS,
                      validation_split=0.15,
                      callbacks=[early_stopping], verbose=1)

        FD_predictions = LSTMIMO.predict(test_inputs['X'])
        FD_eval_df = create_evaluation_df(FD_predictions, test_inputs, HORIZON, y_scaler)
        mae = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAE')
        rmse = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'RMSE')
        #print('rmse {}'.format(rmse))
        metrics.loc[10*i + j] = pd.Series({'mae':mae, 'rmse':rmse, 'B': names[i]})
metrics.to_csv('./results/GEP/LSTM_MIMO10.csv')

Epoch 1/100
112/112 [==============================] - ETA: 0s - loss: 0.0527 - mean_squared_error: 0.0527

KeyboardInterrupt: 

In [1]:
def train_test_split(df, n_test):
    test_df = df.copy()[-(n_test+23):]
    train_df = df.copy()[:-(len(test_df)-23)]
    return train_df, test_df

In [3]:
def ConditionalRNN_data_preparation(df, n_test=4380, T=24, HORIZON=24):
    df = src.preprocessing.preprocess(df, 'Belgium')
    c2 = series_to_supervised(df)
    c1 = df.iloc[:,1:9]
    c1 = c1.loc[c2.index]
    short_df = df.iloc[:,[0,1,-4,-3,-2,-1]].copy()
    #check how to formulate more correctly
    short_df = short_df[146:]
    train_df, test_df = train_test_split(short_df, n_test)
    train_c1, test_c1 = train_test_split(c1, n_test)
    train_c2, test_c2 = train_test_split(c2, n_test)
    y_scaler = MinMaxScaler()
    y_scaler.fit(train_df[['value']])
    train_c2[['value(t-24)']] = y_scaler.fit_transform(train_c2[['value(t-24)']])
    train_c2[['value(t-168)']] = y_scaler.fit_transform(train_c2[['value(t-168)']])
    X_scaler = MinMaxScaler()
    train_df[train_df.columns] = X_scaler.fit_transform(train_df)
    test_df[train_df.columns] = X_scaler.fit_transform(test_df)
    tensor_structure = {'X':(range(-T+1, 1), ['value','fractional hour_sin','fractional hour_cos','day of year_sin','day of year_cos'])}
    train_inputs = TimeSeriesTensor(train_df, 'value', HORIZON, tensor_structure)
    test_inputs = TimeSeriesTensor(test_df, 'value', HORIZON, tensor_structure)
    train_c1 = c1.reindex(train_inputs.dataframe.index)
    train_c2 = c2.reindex(train_inputs.dataframe.index)
    test_c1 = c1.reindex(test_inputs.dataframe.index)
    test_c2 = c2.reindex(test_inputs.dataframe.index)
    return train_inputs, test_inputs, train_c1, test_c1, train_c2, test_c2, y_scaler

In [4]:
def MIMO_fulldata_preparation(df, n_test=4380, T=24, HORIZON=24):
    df = src.preprocessing.preprocess(df, 'Belgium')
    df = df.merge(series_to_supervised(df), how='right', left_index=True, right_index=True)
    train_df, test_df = train_test_split(df, n_test)
    y_scaler = MinMaxScaler()
    y_scaler.fit(train_df[['value']])
    long_scaler = MinMaxScaler()
    test_df[test_df.columns] = long_scaler.fit_transform(test_df)
    train_df[train_df.columns] = long_scaler.fit_transform(train_df)
    tensor_structure = {'X':(range(-T+1, 1), train_df.columns)}
    train_inputs = TimeSeriesTensor(train_df, 'value', HORIZON, tensor_structure)
    test_inputs = TimeSeriesTensor(test_df, 'value', HORIZON, tensor_structure)
    return train_inputs, test_inputs, y_scaler

In [6]:
def MIMO_fulldataT_preparation(df, n_test=4380, T=24, HORIZON=24):
    df = src.preprocessing.preprocess(df, 'Belgium')
    df = df.merge(series_to_supervised(df), how='right', left_index=True, right_index=True)
    df = df.merge(temp, how='left', left_index=True, right_index=True)
    train_df, test_df = train_test_split(df, n_test)
    y_scaler = MinMaxScaler()
    y_scaler.fit(train_df[['value']])
    long_scaler = MinMaxScaler()
    test_df[test_df.columns] = long_scaler.fit_transform(test_df)
    train_df[train_df.columns] = long_scaler.fit_transform(train_df)
    tensor_structure = {'X':(range(-T+1, 1), train_df.columns)}
    train_inputs = TimeSeriesTensor(train_df, 'value', HORIZON, tensor_structure)
    test_inputs = TimeSeriesTensor(test_df, 'value', HORIZON, tensor_structure)
    return train_inputs, test_inputs, y_scaler

In [38]:
temp = pd.read_csv('../data/GEP/Temp_15min.csv', index_col=0, header=0, names=['temp'])